# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data. This will give you a sense of the data you'll be working with.

In [ ]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [ ]:
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function

    return None, None

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different work ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [ ]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function

    return None

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
In this section you'll build the components necessary to build an RNN by implementing the RNN Module and Forward and back propigation function.

### Check the Version of PyTorch and Access to GPU

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import torch

# Check PyTorch Version
minimum_torch_version = '0.3'
assert LooseVersion(torch.__version__) >= LooseVersion(minimum_torch_version), \
    'Please use PyTorch version {} or newer'.format(minimum_torch_version)
print('PyTorch Version: {}'.format(torch.__version__))

# Check for a GPU
if not torch.cuda.is_available():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')

### Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide an api to our dataset. Combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

In [ ]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch
    :return: DataLoader with batched data
    """
    # TODO: Implement function
    
    return None

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_batch_data(batch_data)

### Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). To do this, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `forward` - Forward propigation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propigation function will use these layers to run forward propigation.

In [ ]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, output_size, sequence_length):
        """
        Initialize the PyTorch Module
        :param input_size: The number of input dimensions of the neural network
        :param output_size: The number of output dimensions of the neural network
        :param sequence_length: The sequence length of each batch
        """
        super(RNN, self).__init__()
        # TODO: Implement function
        
        pass

    def forward(self, nn_input):
        """
        Forwad propigation on the neural network
        :param nn_input: The input to the neural network
        :return: The output of the neural network
        """
        # TODO: Implement function
        
        return None


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN)

### Build back propigation
Use the RNN class you implemented to apply forward and back propigation. You'll use the `RNN` class in this function like any other PyTorch Module.

In [ ]:
def forward_back_prop(decoder, decoder_optimizer, criterion, inp, target):
    """
    Forwad and backward propigation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss
    """
    # TODO: Implement Function

    return None


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(forward_back_prop)

## Neural Network Training
With the structue of the network complete and data ready to be fed in the neural network, it's time to train it.
### Train Loop
The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. It's progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other paramters in the next section.

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from torch.autograd import Variable

def train_decoder(batched_data, decoder, decoder_optimizer, criterion, n_epochs, show_every_n_batches):
    chunk_losses = []

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        for batch_i, (inp, target) in enumerate(batched_data, 1):
            loss = forward_back_prop(decoder, decoder_optimizer, criterion, Variable(inp), Variable(target))
            chunk_losses.append(loss)

            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4} Current Batch: {:>6.2f}% Loss: {}\n'.format(
                    epoch_i, n_epochs, batch_i/len(batched_data) * 100, np.average(chunk_losses)))
                chunk_losses = []

    return decoder

### Hyperparameters
Set and train the neural network with the following parameters:
- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

If the network isn't getting the desiered results, tweak these parameters and/or the layers in the `RNN` class.

In [ ]:
# Number of Epochs
num_epochs = None
# Batch Size
batch_size = None
# Sequence Length
sequence_length = None
# Learning Rate
learning_rate = None
# Show stats for every n number of batches
show_every_n_batches = 100

### Train
In the next cell, you'll train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the similar problems.

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
vocab_size = len(vocab_to_int)

decoder = RNN(vocab_size, vocab_size, sequence_length)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
batched_data = batch_data(int_text, sequence_length, batch_size)
trained_decoder = train_decoder(batched_data, decoder, decoder_optimizer, criterion, num_epochs, show_every_n_batches)

helper.save_model('./save/trained_decoder', trained_decoder)
print('Model Trained and Saved')

# Checkpoint

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_decoder = helper.load_model('./save/trained_decoder')

## Generate TV Script
With the network trained and saved, you'll use it to generate a Simpsons TV script in this section.
### Generate Text
To generate the text, the network needs to start with a single word and repeat it's predictions untill it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and genereates a set length of text, `predict_len`.

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
def generate(decoder, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    current_seq = np.full((1, decoder.sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        output = decoder(Variable(torch.LongTensor(current_seq)))
        top_i = torch.multinomial(output.view(-1).exp().data, 1)[0]
        word = int_to_vocab[top_i]
        
        predicted.append(word)
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = top_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
        
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "homer_simpson"
- "moe_szyslak"
- "Barney_Gumble"

In [ ]:
gen_length = 200
prime_word = 'moe_szyslak' # homer_simpson, moe_szyslak, or Barney_Gumble

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
print(generate(trained_decoder, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length))

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.